# Day Fourteen

## Task

The [task](https://adventofcode.com/2017/day/14) today is to compute some more hashes.

### Part One

First, some crazy business.

In [1]:
def add_row_index(string, row_index):
    return string + str(row_index)


assert add_row_index('abc-', 0) == 'abc-0'
assert add_row_index('abc-', 127) == 'abc-127'


d = {
    '0': '0000', '1': '0001', '2': '0010', '3': '0011',
    '4': '0100', '5': '0101', '6': '0110', '7': '0111',
    '8': '1000', '9': '1001', 'a': '1010', 'b': '1011',
    'c': '1100', 'd': '1101', 'e': '1110', 'f': '1111',
}
def hex_to_binary_str(hex_str):
    return ''.join(map(d.get, hex_str))


assert hex_to_binary_str('0') == '0000'
assert hex_to_binary_str('0f') == '00001111'
assert hex_to_binary_str('9a') == '10011010'

Second, let's bring back our hashing function, `output`.

In [2]:
from collections import deque
from copy import copy
from functools import reduce
from itertools import islice


def twist(sequence, position, skip, length):
    s = copy(sequence)
    
    s.rotate(-position)
    s = deque(
        list(reversed(list(islice(s, 0, length))))
        + list(islice(s, length, len(s)))
    )
    s.rotate(position)
    
    return s


def hash_(sequence, lengths, position=0, skip=0):
    s = deque(sequence)
    position_ = position
    skip_ = skip
    
    for length in lengths:
        s = twist(s, position_, skip_, length)
        position_ = (position_ + skip_ + length) % len(s)
        skip_ += 1
    
    return list(s), position_, skip_

def encode(string):
    return ','.join([str(ord(x)) for x in string] + ['17', '31', '73', '47', '23'])


def cycle(sequence, lengths):
    s = copy(sequence)
    position = 0
    skip = 0
    
    for _ in range(0, 64):
        s, position, skip = hash_(s, lengths, position, skip)

    return s


def output(string):
    sequence = range(0, 256)
    lengths = [int(x) for x in encode(string.strip()).split(',')]
    sparse_hash = cycle(range(0, 256), lengths)
    dense_hash = map(
        lambda x: reduce(lambda y, z: y ^ z, x), 
        [sparse_hash[x:x+16] for x in range(0, len(sparse_hash), 16)]
    )

    return ''.join([hex(x)[2:].zfill(2) for x in dense_hash])

Now let's join it up.

In [3]:
def read_example():
    return 'flqrgnkx'


def compute_grid(string):
    rows = map(lambda x: add_row_index(string + '-', x), range(0, 128))
    rows = map(output, rows)
    return map(hex_to_binary_str, rows)
    

def count_(string):
    rows = compute_grid(string)

    return sum(map(lambda x: x.count('1'), rows))


assert count_(read_example()) == 8108

And on the puzzle, we have

In [4]:
def read_puzzle():
    return 'amgozmfv'

count_(read_puzzle())

8222

### Part Two

For the second part, we need to find the number of distinct groups in the grid. For this I'm going to create a graph and then using the work done on [day twelve](https://github.com/jdgillespie91/aoc/blob/master/aoc/day12/investigation.ipynb). First, we need to extend that work to count single nodes as distinct groups.

In [5]:
from queue import Empty, Queue


def search(graph, start=0):
    queue = Queue()
    queue.put(start)
    discovered = set()
    
    while True:
        try:
            node = queue.get(timeout=0)
        except Empty:
            break
        else:
            for neighbour in (graph[node] - discovered):
                queue.put(neighbour)
                discovered.add(neighbour)

    return discovered


def number_of_groups(graph):
    groups = len({k for k, v in graph.items() if v == set()})
    graph = {k: v for k, v in graph.items() if v != set()}
    
    while graph:
        nodes = search(graph, start=next(iter(graph)))
        
        for node in nodes:
            del graph[node]
            
        groups += 1
        
    return groups

Now let's translate our grid into a searchable graph.

In [6]:
from collections import defaultdict


def compute_coordinates(grid):
    coordinates = set()
    
    for j, row in enumerate(grid):
        for i, char in enumerate(row):
            if char == '1':
                coordinates.add((i, j))
                
    return coordinates


def compute_graph_dict(coordinates):
    graph = defaultdict(set)
    
    for coordinate in coordinates:
        graph[coordinate]
        
        if (coordinate[0], coordinate[1] + 1) in coordinates:
            graph[coordinate].add((coordinate[0], coordinate[1] + 1))
            graph[(coordinate[0], coordinate[1] + 1)].add(coordinate)
        
        if (coordinate[0] + 1, coordinate[1]) in coordinates:
            graph[coordinate].add((coordinate[0] + 1, coordinate[1]))
            graph[(coordinate[0] + 1, coordinate[1])].add(coordinate)
        
    return graph


def compute_graph(grid):
    coordinates = compute_coordinates(grid)
    return compute_graph_dict(coordinates)

number_of_groups(compute_graph(list(compute_grid(read_puzzle()))))

1086